In [2]:
import os
os.environ["GOOGLE_API_KEY"] = ""
os.environ["GEMINI_API_KEY"] = ""

In [3]:
# Install aiohttp if not present
!pip install aiohttp -q

In [4]:
import os
import asyncio
import aiohttp
import json
import time
import statistics
from dataclasses import dataclass, field
from typing import Any, List, Optional, TypedDict, Dict, Callable, Coroutine
from contextlib import asynccontextmanager
from functools import wraps
from langchain_core.prompts import PromptTemplate
from langchain_core.language_models.llms import BaseLLM
from langchain_core.outputs import LLMResult
from langchain_core.callbacks.manager import AsyncCallbackManagerForLLMRun, CallbackManagerForLLMRun
from pydantic import Field
from langgraph.graph import StateGraph, END

## Async Instrumentation Module

This module provides comprehensive latency and throughput measurement for async LangGraph agents.

In [5]:
# --- ASYNC INSTRUMENTATION CODE ---

@dataclass
class TimingRecord:
    """Single timing measurement record."""
    name: str
    start_time: float
    end_time: float
    duration_ms: float
    metadata: Dict[str, Any] = field(default_factory=dict)


@dataclass
class LLMCallMetrics:
    """Metrics for a single LLM API call."""
    latency_ms: float
    input_chars: int
    output_chars: int
    model_name: str
    success: bool
    is_async: bool = True
    error_message: Optional[str] = None


class AsyncInstrumentationCollector:
    """Collects and aggregates timing/throughput metrics for async operations."""

    def __init__(self):
        self.timing_records: List[TimingRecord] = []
        self.llm_metrics: List[LLMCallMetrics] = []
        self.node_timings: Dict[str, List[float]] = {}
        self.graph_start_time: Optional[float] = None
        self.graph_end_time: Optional[float] = None
        self._run_count: int = 0
        self._lock = asyncio.Lock() if asyncio.get_event_loop().is_running() else None

    def _ensure_lock(self):
        """Ensure lock exists for thread safety in async context."""
        if self._lock is None:
            try:
                self._lock = asyncio.Lock()
            except RuntimeError:
                pass

    def reset(self):
        """Reset all collected metrics for a new run."""
        self.timing_records = []
        self.llm_metrics = []
        self.node_timings = {}
        self.graph_start_time = None
        self.graph_end_time = None

    def start_graph(self):
        """Mark the start of graph execution."""
        self.graph_start_time = time.perf_counter()
        self._run_count += 1

    def end_graph(self):
        """Mark the end of graph execution."""
        self.graph_end_time = time.perf_counter()

    @asynccontextmanager
    async def measure_async(self, name: str, **metadata):
        """Async context manager to measure execution time of an async code block."""
        start = time.perf_counter()
        try:
            yield
        finally:
            end = time.perf_counter()
            duration_ms = (end - start) * 1000
            record = TimingRecord(
                name=name,
                start_time=start,
                end_time=end,
                duration_ms=duration_ms,
                metadata=metadata
            )
            self.timing_records.append(record)

            if name not in self.node_timings:
                self.node_timings[name] = []
            self.node_timings[name].append(duration_ms)

    def record_llm_call(self, metrics: LLMCallMetrics):
        """Record metrics from an LLM API call."""
        self.llm_metrics.append(metrics)

    def get_total_latency_ms(self) -> float:
        """Get total graph execution latency in milliseconds."""
        if self.graph_start_time and self.graph_end_time:
            return (self.graph_end_time - self.graph_start_time) * 1000
        return 0.0

    def get_llm_latency_stats(self) -> Dict[str, float]:
        """Get statistics on LLM call latencies."""
        if not self.llm_metrics:
            return {}

        latencies = [m.latency_ms for m in self.llm_metrics]
        return {
            "count": len(latencies),
            "total_ms": sum(latencies),
            "mean_ms": statistics.mean(latencies),
            "min_ms": min(latencies),
            "max_ms": max(latencies),
            "median_ms": statistics.median(latencies),
            "stdev_ms": statistics.stdev(latencies) if len(latencies) > 1 else 0.0,
            "async_calls": sum(1 for m in self.llm_metrics if m.is_async)
        }

    def get_throughput_stats(self) -> Dict[str, float]:
        """Calculate throughput metrics."""
        total_time_sec = self.get_total_latency_ms() / 1000
        if total_time_sec == 0:
            return {}

        total_input_chars = sum(m.input_chars for m in self.llm_metrics)
        total_output_chars = sum(m.output_chars for m in self.llm_metrics)
        num_requests = len(self.llm_metrics)

        est_input_tokens = total_input_chars / 4
        est_output_tokens = total_output_chars / 4

        return {
            "requests_per_second": num_requests / total_time_sec,
            "input_chars_per_second": total_input_chars / total_time_sec,
            "output_chars_per_second": total_output_chars / total_time_sec,
            "est_input_tokens_per_second": est_input_tokens / total_time_sec,
            "est_output_tokens_per_second": est_output_tokens / total_time_sec,
            "total_input_chars": total_input_chars,
            "total_output_chars": total_output_chars,
            "est_total_input_tokens": int(est_input_tokens),
            "est_total_output_tokens": int(est_output_tokens)
        }

    def get_node_breakdown(self) -> Dict[str, Dict[str, float]]:
        """Get timing breakdown by node."""
        breakdown = {}
        for name, timings in self.node_timings.items():
            breakdown[name] = {
                "count": len(timings),
                "total_ms": sum(timings),
                "mean_ms": statistics.mean(timings),
                "min_ms": min(timings),
                "max_ms": max(timings)
            }
        return breakdown

    def print_report(self):
        """Print a formatted instrumentation report."""
        print("\n" + "="*60)
        print("ASYNC INSTRUMENTATION REPORT")
        print("="*60)

        print(f"\n📊 TOTAL LATENCY")
        print(f"   Total Graph Execution: {self.get_total_latency_ms():.2f} ms")

        print(f"\n📍 NODE-LEVEL BREAKDOWN")
        breakdown = self.get_node_breakdown()
        for node_name, stats in breakdown.items():
            print(f"   {node_name}:")
            print(f"      - Total: {stats['total_ms']:.2f} ms")
            print(f"      - Mean:  {stats['mean_ms']:.2f} ms")

        print(f"\n🤖 ASYNC LLM CALL STATISTICS")
        llm_stats = self.get_llm_latency_stats()
        if llm_stats:
            print(f"   Number of calls:  {llm_stats['count']}")
            print(f"   Async calls:      {llm_stats['async_calls']}")
            print(f"   Total LLM time:   {llm_stats['total_ms']:.2f} ms")
            print(f"   Mean latency:     {llm_stats['mean_ms']:.2f} ms")
            print(f"   Min latency:      {llm_stats['min_ms']:.2f} ms")
            print(f"   Max latency:      {llm_stats['max_ms']:.2f} ms")
            print(f"   Median latency:   {llm_stats['median_ms']:.2f} ms")

        print(f"\n🚀 THROUGHPUT METRICS")
        throughput = self.get_throughput_stats()
        if throughput:
            print(f"   Requests/second:       {throughput['requests_per_second']:.4f}")
            print(f"   Output chars/second:   {throughput['output_chars_per_second']:.2f}")
            print(f"   Est. output tokens/s:  {throughput['est_output_tokens_per_second']:.2f}")
            print(f"   Total input chars:     {throughput['total_input_chars']}")
            print(f"   Total output chars:    {throughput['total_output_chars']}")

        print(f"\n⏱️  EXECUTION TIMELINE")
        for record in self.timing_records:
            print(f"   [{record.duration_ms:8.2f} ms] {record.name}")

        print("\n" + "="*60)

    def to_dict(self) -> Dict[str, Any]:
        """Export all metrics as a dictionary."""
        return {
            "total_latency_ms": self.get_total_latency_ms(),
            "llm_statistics": self.get_llm_latency_stats(),
            "throughput": self.get_throughput_stats(),
            "node_breakdown": self.get_node_breakdown(),
            "run_count": self._run_count,
            "timeline": [
                {"name": r.name, "duration_ms": r.duration_ms, "metadata": r.metadata}
                for r in self.timing_records
            ]
        }


# Global collector instance
collector = AsyncInstrumentationCollector()


def instrumented_async_node(name: str):
    """Decorator to instrument an async LangGraph node function."""
    def decorator(func: Callable):
        @wraps(func)
        async def wrapper(state, *args, **kwargs):
            async with collector.measure_async(name):
                return await func(state, *args, **kwargs)
        return wrapper
    return decorator

In [6]:
# --- 1. Async Custom LLM Implementation ---

class AsyncCustomHTTPGemini(BaseLLM):
    """
    An async custom LangChain LLM wrapper that interacts with the Google Gemini API
    using aiohttp for asynchronous HTTP requests.
    """

    api_key: Optional[str] = None
    model_name: str = Field(default="gemini-2.5-flash", alias="model")
    base_url: str = "https://generativelanguage.googleapis.com/v1beta/models/"
    response_schema: Optional[dict] = None
    _session: Optional[aiohttp.ClientSession] = None

    class Config:
        arbitrary_types_allowed = True

    def __init__(self, **kwargs: Any):
        super().__init__(**kwargs)
        if not self.api_key:
            self.api_key = os.getenv("GEMINI_API_KEY")
        if not self.api_key:
            raise ValueError("GEMINI_API_KEY must be provided or set as an environment variable.")

    @property
    def _llm_type(self) -> str:
        return "async_custom_http_gemini"

    async def _get_session(self) -> aiohttp.ClientSession:
        """Get or create an aiohttp session."""
        if self._session is None or self._session.closed:
            self._session = aiohttp.ClientSession()
        return self._session

    async def _acall(
        self,
        prompt: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[AsyncCallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> str:
        """
        Async HTTP POST request to the Gemini API with instrumentation.
        """
        api_endpoint = f"{self.base_url}{self.model_name}:generateContent"
        url = f"{api_endpoint}?key={self.api_key}"
        headers = {"Content-Type": "application/json"}

        request_data = {
            "contents": [{"parts": [{"text": prompt}]}]
        }

        if self.response_schema:
            request_data["generationConfig"] = {
                "responseMimeType": "application/json",
                "responseSchema": self.response_schema
            }

        input_chars = len(prompt)
        start_time = time.perf_counter()

        try:
            session = await self._get_session()
            async with session.post(url, headers=headers, json=request_data) as response:
                response.raise_for_status()
                response_json = await response.json()
                generated_text = response_json['candidates'][0]['content']['parts'][0]['text']

                end_time = time.perf_counter()
                latency_ms = (end_time - start_time) * 1000

                # Record async metrics
                collector.record_llm_call(LLMCallMetrics(
                    latency_ms=latency_ms,
                    input_chars=input_chars,
                    output_chars=len(generated_text),
                    model_name=self.model_name,
                    success=True,
                    is_async=True
                ))

                return generated_text

        except aiohttp.ClientResponseError as err:
            end_time = time.perf_counter()
            latency_ms = (end_time - start_time) * 1000

            collector.record_llm_call(LLMCallMetrics(
                latency_ms=latency_ms,
                input_chars=input_chars,
                output_chars=0,
                model_name=self.model_name,
                success=False,
                is_async=True,
                error_message=str(err)
            ))

            error_message = f"Gemini API HTTP Error ({err.status}): {err.message}"
            raise RuntimeError(error_message) from err
        except Exception as e:
            raise RuntimeError(f"An unexpected error occurred during async API call: {e}")

    def _call(
        self,
        prompt: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> str:
        """Sync fallback - runs the async version in an event loop."""
        return asyncio.get_event_loop().run_until_complete(
            self._acall(prompt, stop, None, **kwargs)
        )

    async def _agenerate(
        self,
        prompts: List[str],
        stop: Optional[List[str]] = None,
        run_manager: Optional[AsyncCallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> LLMResult:
        """Async generation for multiple prompts."""
        generations = []
        for prompt in prompts:
            text = await self._acall(prompt, stop, run_manager, **kwargs)
            generations.append([{"text": text}])
        return LLMResult(generations=generations)

    def _generate(
        self,
        prompts: List[str],
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> LLMResult:
        """Sync generate fallback."""
        generations = []
        for prompt in prompts:
            text = self._call(prompt, stop, run_manager, **kwargs)
            generations.append([{"text": text}])
        return LLMResult(generations=generations)

    async def close(self):
        """Close the aiohttp session."""
        if self._session and not self._session.closed:
            await self._session.close()

/tmp/ipython-input-1993174299.py:3: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  class AsyncCustomHTTPGemini(BaseLLM):


In [7]:
# --- 2. Async LangGraph AGENT DEFINITIONS ---

# 2.1. Define the State for the Graph
class AgentState(TypedDict):
    item_name: str
    complexity_level: str
    score: Optional[float]
    review_data: Optional[str]


# --- JSON Schema Definitions ---
SCORE_SCHEMA = {
    "type": "OBJECT",
    "properties": {
        "score": {"type": "NUMBER", "description": "A random technical score between 0.0 and 1.0."},
    },
    "required": ["score"],
    "propertyOrdering": ["score"]
}

REVIEW_SCHEMA = {
    "type": "OBJECT",
    "properties": {
        "review_text": {"type": "STRING", "description": "A concise, technical review."},
        "category": {"type": "STRING", "description": "The category of the review (e.g., 'Positive', 'Neutral', 'Negative')."}
    },
    "required": ["review_text", "category"],
    "propertyOrdering": ["review_text", "category"]
}


# 2.2. Async Node for Agent 1: Score Generator
@instrumented_async_node("score_generator_node")
async def score_generator_node(state: AgentState) -> dict:
    """Async: Generates a technical score (0.0 to 1.0) for the item."""
    print("--- [Agent 1] Executing: Score Generator (ASYNC) ---")

    llm_score_generator = AsyncCustomHTTPGemini(model_name="gemini-2.5-flash", response_schema=SCORE_SCHEMA)

    prompt_1 = PromptTemplate.from_template(
        "You are a technical analyst. Your task is to assign a random score between 0.0 and 1.0 to the '{item_name}' based on its complexity '{complexity_level}'. Output the result strictly in JSON format according to the schema."
    )
    prompt_value = prompt_1.format(
        item_name=state['item_name'],
        complexity_level=state['complexity_level']
    )

    # Use async invoke
    raw_json_output = await llm_score_generator.ainvoke(prompt_value)
    await llm_score_generator.close()  # Clean up session

    try:
        score_data = json.loads(raw_json_output)
        score = score_data.get('score', 0.0)
        print(f"--- [Agent 1] Generated Score: {score} ---")
        return {"score": score}
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON from Agent 1: {e}. Falling back to score 0.5.")
        return {"score": 0.5}


# 2.3. Async Node for Agent 2: Review Generator
@instrumented_async_node("review_generator_node")
async def review_generator_node(state: AgentState) -> dict:
    """Async: Generates a detailed review based on the generated score and context."""
    print("\n--- [Agent 2] Executing: Review Generator (ASYNC) ---")

    llm_review_generator = AsyncCustomHTTPGemini(model_name="gemini-2.5-flash", response_schema=REVIEW_SCHEMA)

    prompt_2 = PromptTemplate.from_template(
        "Generate a technical review for the item '{item_name}' which has a complexity of '{complexity_level}' and received a technical score of {score}. Your review must reflect this score. Output the review strictly in JSON format according to the schema."
    )

    prompt_value = prompt_2.format(
        item_name=state['item_name'],
        complexity_level=state['complexity_level'],
        score=state['score']
    )

    # Use async invoke
    raw_json_output = await llm_review_generator.ainvoke(prompt_value)
    await llm_review_generator.close()  # Clean up session

    print("--- [Agent 2] Generated Review Data ---")
    return {"review_data": raw_json_output}

In [8]:
# --- 2.4. Async LangGraph Setup and Execution ---

async def run_async_graph():
    """Main async function to build and run the graph."""
    print("--- LangGraph Async Custom HTTP Gemini Cascading Agent Example ---")
    print("NOTE: With ASYNC Instrumentation for Latency & Throughput Measurement")

    if not os.getenv("GEMINI_API_KEY"):
        print("\nERROR: GEMINI_API_KEY environment variable not set.")
        print("Please set the GEMINI_API_KEY environment variable and try again.")
        return None

    # --- Build the Graph ---
    graph_builder = StateGraph(AgentState)
    graph_builder.add_node("score_generator", score_generator_node)
    graph_builder.add_node("review_generator", review_generator_node)
    graph_builder.set_entry_point("score_generator")
    graph_builder.add_edge("score_generator", "review_generator")
    graph_builder.add_edge("review_generator", END)
    app = graph_builder.compile()

    # --- Example Invocation with Async Instrumentation ---
    print("\nInvoking the two-agent LangGraph application (ASYNC)...")

    initial_state = {
        "item_name": "Quantum Entanglement Module v1.2",
        "complexity_level": "High/Experimental"
    }

    # Reset collector and start timing
    collector.reset()
    collector.start_graph()

    # Run the graph asynchronously
    final_state = await app.ainvoke(initial_state)

    # End timing
    collector.end_graph()

    final_review_json = final_state.get('review_data')
    final_score = final_state.get('score')

    print("\n--- Graph Execution Complete (ASYNC) ---")
    print(f"Original Input:\n{json.dumps(initial_state, indent=2)}")
    print(f"Intermediate Score Generated by Agent 1: {final_score}")

    print("\n--- Final Output (Generated by Agent 2) ---")
    try:
        print(json.dumps(json.loads(final_review_json), indent=2))
    except json.JSONDecodeError:
        print(f"Raw Output:\n{final_review_json}")

    # --- Print Instrumentation Report ---
    collector.print_report()

    return app, final_state


# Run the async graph
# In Jupyter, we can use await directly
result = await run_async_graph()

--- LangGraph Async Custom HTTP Gemini Cascading Agent Example ---
NOTE: With ASYNC Instrumentation for Latency & Throughput Measurement

Invoking the two-agent LangGraph application (ASYNC)...
--- [Agent 1] Executing: Score Generator (ASYNC) ---
--- [Agent 1] Generated Score: 0.732418 ---

--- [Agent 2] Executing: Review Generator (ASYNC) ---
--- [Agent 2] Generated Review Data ---

--- Graph Execution Complete (ASYNC) ---
Original Input:
{
  "item_name": "Quantum Entanglement Module v1.2",
  "complexity_level": "High/Experimental"
}
Intermediate Score Generated by Agent 1: 0.732418

--- Final Output (Generated by Agent 2) ---
{
  "review_text": "The Quantum Entanglement Module v1.2 demonstrates promising entanglement fidelity, achieving a quantum state purity of 0.88 +/- 0.02 under controlled cryogenic conditions. Interfacing with existing quantum computing architectures remains experimental, exhibiting occasional decoherence issues during state transfer. While the high complexity wa

## Async Batch Throughput Testing

Run multiple concurrent iterations to measure true async throughput benefits.

In [9]:
async def run_single_invocation(app, test_case: Dict, run_id: int) -> Dict:
    """Run a single graph invocation and return metrics."""
    local_collector = AsyncInstrumentationCollector()

    local_collector.reset()
    local_collector.start_graph()

    try:
        _ = await app.ainvoke(test_case)
        local_collector.end_graph()
        return {
            "run_id": run_id,
            "success": True,
            "latency_ms": local_collector.get_total_latency_ms(),
            "test_case": test_case.get('item_name', 'Unknown')
        }
    except Exception as e:
        local_collector.end_graph()
        return {
            "run_id": run_id,
            "success": False,
            "latency_ms": local_collector.get_total_latency_ms(),
            "error": str(e),
            "test_case": test_case.get('item_name', 'Unknown')
        }


async def run_async_batch_throughput_test(
    test_cases: List[Dict],
    num_iterations: int = 3,
    concurrent: bool = True
):
    """
    Run async batch throughput test.

    Args:
        test_cases: List of initial states to test
        num_iterations: Number of times to run each test case
        concurrent: If True, run iterations concurrently within each test case
    """
    print(f"\n{'='*60}")
    print(f"ASYNC BATCH THROUGHPUT TEST")
    print(f"Running {len(test_cases)} test cases x {num_iterations} iterations")
    print(f"Mode: {'CONCURRENT' if concurrent else 'SEQUENTIAL'}")
    print(f"{'='*60}")

    # Build the graph once
    graph_builder = StateGraph(AgentState)
    graph_builder.add_node("score_generator", score_generator_node)
    graph_builder.add_node("review_generator", review_generator_node)
    graph_builder.set_entry_point("score_generator")
    graph_builder.add_edge("score_generator", "review_generator")
    graph_builder.add_edge("review_generator", END)
    app = graph_builder.compile()

    all_results = []
    total_start = time.perf_counter()

    run_id = 0
    for i, test_case in enumerate(test_cases):
        print(f"\nTest case {i+1}: {test_case.get('item_name', 'Unknown')}")

        if concurrent:
            # Run iterations concurrently
            tasks = [
                run_single_invocation(app, test_case, run_id + j)
                for j in range(num_iterations)
            ]
            results = await asyncio.gather(*tasks)
            all_results.extend(results)

            for r in results:
                status = "OK" if r['success'] else f"ERROR: {r.get('error', 'unknown')}"
                print(f"  Run {r['run_id']}: {r['latency_ms']:.2f} ms - {status}")
        else:
            # Run iterations sequentially
            for j in range(num_iterations):
                result = await run_single_invocation(app, test_case, run_id + j)
                all_results.append(result)
                status = "OK" if result['success'] else f"ERROR: {result.get('error', 'unknown')}"
                print(f"  Iteration {j+1}: {result['latency_ms']:.2f} ms - {status}")

        run_id += num_iterations

    total_end = time.perf_counter()
    total_time_sec = total_end - total_start

    # Aggregate statistics
    print(f"\n{'='*60}")
    print("AGGREGATE RESULTS")
    print(f"{'='*60}")

    successful_results = [r for r in all_results if r['success']]
    latencies = [r['latency_ms'] for r in successful_results]

    print(f"\nTotal runs attempted: {len(all_results)}")
    print(f"Successful runs: {len(successful_results)}")
    print(f"Total wall-clock time: {total_time_sec:.2f} seconds")

    if latencies:
        print(f"\nEnd-to-end Latency (per run):")
        print(f"  Mean:   {statistics.mean(latencies):.2f} ms")
        print(f"  Median: {statistics.median(latencies):.2f} ms")
        print(f"  Min:    {min(latencies):.2f} ms")
        print(f"  Max:    {max(latencies):.2f} ms")
        if len(latencies) > 1:
            print(f"  Stdev:  {statistics.stdev(latencies):.2f} ms")

        print(f"\nThroughput:")
        print(f"  Runs/second (wall-clock): {len(successful_results) / total_time_sec:.4f}")

        # Calculate theoretical sequential time
        theoretical_sequential = sum(latencies) / 1000
        speedup = theoretical_sequential / total_time_sec if total_time_sec > 0 else 1
        print(f"\nAsync Performance:")
        print(f"  Theoretical sequential time: {theoretical_sequential:.2f} sec")
        print(f"  Actual wall-clock time:      {total_time_sec:.2f} sec")
        print(f"  Speedup factor:              {speedup:.2f}x")

    return all_results

In [13]:


test_cases = [
    {"item_name": "Quantum Entanglement Module v1.2", "complexity_level": "High/Experimental"},
    {"item_name": "Basic Data Logger", "complexity_level": "Low/Simple"},
    {"item_name": "ML Inference Pipeline", "complexity_level": "Medium/Standard"},
]

# Run concurrent test
concurrent_results = await run_async_batch_throughput_test(test_cases, num_iterations=3, concurrent=True)

# Run sequential test for comparison
sequential_results = await run_async_batch_throughput_test(test_cases, num_iterations=3, concurrent=False)


ASYNC BATCH THROUGHPUT TEST
Running 3 test cases x 3 iterations
Mode: CONCURRENT

Test case 1: Quantum Entanglement Module v1.2
--- [Agent 1] Executing: Score Generator (ASYNC) ---
--- [Agent 1] Executing: Score Generator (ASYNC) ---
--- [Agent 1] Executing: Score Generator (ASYNC) ---
--- [Agent 1] Generated Score: 0.785 ---

--- [Agent 2] Executing: Review Generator (ASYNC) ---
--- [Agent 1] Generated Score: 0.75 ---

--- [Agent 2] Executing: Review Generator (ASYNC) ---
--- [Agent 1] Generated Score: 0.73 ---

--- [Agent 2] Executing: Review Generator (ASYNC) ---
--- [Agent 2] Generated Review Data ---
  Run 0: 1347.43 ms - ERROR: Gemini API HTTP Error (429): Too Many Requests
  Run 1: 1398.16 ms - ERROR: Gemini API HTTP Error (429): Too Many Requests
  Run 2: 3862.58 ms - OK

Test case 2: Basic Data Logger
--- [Agent 1] Executing: Score Generator (ASYNC) ---
--- [Agent 1] Executing: Score Generator (ASYNC) ---
--- [Agent 1] Executing: Score Generator (ASYNC) ---
  Run 3: 87.72 ms 

## Parallel Node Execution Example

Demonstrates running independent nodes in parallel for maximum async benefit.

In [14]:
# --- Extended State for Parallel Execution ---

class ParallelAgentState(TypedDict):
    """State for parallel agent execution."""
    item_name: str
    complexity_level: str
    score_a: Optional[float]
    score_b: Optional[float]
    combined_result: Optional[str]


@instrumented_async_node("score_generator_a")
async def score_generator_a(state: ParallelAgentState) -> dict:
    """First parallel score generator."""
    print("--- [Agent A] Executing: Score Generator A (ASYNC) ---")
    llm = AsyncCustomHTTPGemini(model_name="gemini-2.5-flash", response_schema=SCORE_SCHEMA)

    prompt = f"Assign a technical innovation score (0-1) for '{state['item_name']}' with complexity '{state['complexity_level']}'. Output JSON."
    result = await llm.ainvoke(prompt)
    await llm.close()

    try:
        score = json.loads(result).get('score', 0.5)
    except:
        score = 0.5
    print(f"--- [Agent A] Score: {score} ---")
    return {"score_a": score}


@instrumented_async_node("score_generator_b")
async def score_generator_b(state: ParallelAgentState) -> dict:
    """Second parallel score generator."""
    print("--- [Agent B] Executing: Score Generator B (ASYNC) ---")
    llm = AsyncCustomHTTPGemini(model_name="gemini-2.5-flash", response_schema=SCORE_SCHEMA)

    prompt = f"Assign a market readiness score (0-1) for '{state['item_name']}' with complexity '{state['complexity_level']}'. Output JSON."
    result = await llm.ainvoke(prompt)
    await llm.close()

    try:
        score = json.loads(result).get('score', 0.5)
    except:
        score = 0.5
    print(f"--- [Agent B] Score: {score} ---")
    return {"score_b": score}


@instrumented_async_node("combiner_node")
async def combiner_node(state: ParallelAgentState) -> dict:
    """Combines results from parallel nodes."""
    print("\n--- [Combiner] Executing: Result Combiner ---")
    avg_score = (state['score_a'] + state['score_b']) / 2
    result = f"Combined Score: {avg_score:.2f} (Innovation: {state['score_a']}, Market: {state['score_b']})"
    print(f"--- [Combiner] {result} ---")
    return {"combined_result": result}


async def run_parallel_graph():
    """Run graph with parallel node execution."""
    print("\n" + "="*60)
    print("PARALLEL NODE EXECUTION EXAMPLE")
    print("="*60)

    # Build parallel graph
    graph_builder = StateGraph(ParallelAgentState)

    # Add nodes
    graph_builder.add_node("score_a", score_generator_a)
    graph_builder.add_node("score_b", score_generator_b)
    graph_builder.add_node("combiner", combiner_node)

    # Set entry point - both score generators run from start
    graph_builder.set_entry_point("score_a")

    # Parallel execution: both A and B can run, then combine
    # Note: LangGraph handles this via edges; for true parallelism,
    # you'd need conditional edges or a fan-out pattern
    graph_builder.add_edge("score_a", "score_b")
    graph_builder.add_edge("score_b", "combiner")
    graph_builder.add_edge("combiner", END)

    app = graph_builder.compile()

    initial_state = {
        "item_name": "Neural Interface Chip",
        "complexity_level": "Very High/Research"
    }

    collector.reset()
    collector.start_graph()

    final_state = await app.ainvoke(initial_state)

    collector.end_graph()

    print(f"\nFinal Result: {final_state.get('combined_result')}")
    collector.print_report()

    return final_state


# Uncomment to run parallel example
# parallel_result = await run_parallel_graph()

## Export Metrics to JSON

In [12]:
def export_metrics_to_json(filename: str = "async_metrics.json"):
    """Export collected metrics to a JSON file."""
    metrics = collector.to_dict()
    with open(filename, 'w') as f:
        json.dump(metrics, f, indent=2)
    print(f"Metrics exported to {filename}")

# Uncomment to export
# export_metrics_to_json("langgraph_async_metrics.json")